In [129]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import numpy as np
import json
import os
from psycopg2.extras import Json
import psycopg2

# Make the request to the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the relevant information
table = soup.find('table', {'class': 'wikitable sortable'})
rows = table.find_all('tr')
data = []
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        symbol = cells[0].text.strip()
        name = cells[1].find('a').get('title').replace(' ','_')
        sector = cells[3].text.strip()
        # Get the Wikipedia page for each company
        company_url = f"https://en.wikipedia.org/wiki/{name}"
        company_response = requests.get(company_url)
        company_soup = BeautifulSoup(company_response.content, 'html.parser')
        #company_text = company_soup.get_text()
        data.append({'Symbol': symbol, 'Name': name, 'Sector': sector, 'Wikipedia Entry': company_soup})
        #set system sleep to 1 second to avoid overloading the server
        time.sleep(0.5)

# Create a Dataframe
df = pd.DataFrame(data)


In [150]:
m=df.copy()
#df = m.copy()
#df.to_csv('SP500_Wikipedia.csv')
#df =pd.read_csv('C:/Users/gabri/OneDrive/Desktop/FH_Stuff/DSI/SP500_Wikipedia.csv')

,Symbol,Name,Sector,Wikipedia Entry
0,MMM,3M,Industrials,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
1,AOS,A._O._Smith,Industrials,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
2,ABT,Abbott_Laboratories,Health Care,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
3,ABBV,AbbVie,Health Care,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
4,ACN,Accenture,Information Technology,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
...,...,...,...,...
498,YUM,Yum!_Brands,Consumer Discretionary,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
499,ZBRA,Zebra_Technologies,Information Technology,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
500,ZBH,Zimmer_Biomet,Health Care,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."
501,ZION,Zions_Bancorporation,Financials,"[html, \n, [\n, [\n, <meta charset=""utf-8""/>, ..."


In [158]:
Dataframes = []
for t in range(0, len(df)):   
    try:
        ############################################### Read Table from Wikipedia Page ###############################################
        comp = df.iloc[t]['Wikipedia Entry']

        #find all tables in the wikipedia page
        tables = comp.find_all('table')[0]
        #find the table with the class 'infobox'
        infobox = comp.find('table', {'class': 'infobox'})

        #find  class 'infobox-label' in the infobox
        infobox_label = infobox.find_all('th', {'class': 'infobox-label'})

        #find  class 'infobox-data' in the infobox
        infobox_data = infobox.find_all('td', {'class': 'infobox-data'})

        #label text
        infobox_label_text = [i.text for i in infobox_label]

        data_vec =[]
        #loop through all entries in the infobox
        for i in range(len(infobox_data)):
            if len(infobox_data[i].find_all('div', {'class': 'plainlist'})) > 0:
                l = infobox_data[i].find_all('div', {'class': 'plainlist'})
                l=[j.text for j in l[0].find_all('li')]
                data_vec.append(l)
            else:
                data_vec.append(infobox_data[i].text)

        DF_n = pd.DataFrame({'label': infobox_label_text, 'data': data_vec})

        ############################################### Clean Data Table ###############################################
        
        #founded
        if ((DF_n['label']=='Founded').any()) and not isinstance(DF_n.loc[int(np.where(DF_n['label']=='Founded')[0])]['data'],list):
            string = DF_n[DF_n['label']=='Founded']['data'].values[0]
            #stringf = DF_n[DF_n['label']=='Founded']['data'].values[0]
            DF_n.loc[DF_n['label']=='Founded','data'] = re.findall(r'\d{4}', string)[0]
            #print(re.findall(r'\d{4}', string.values[0])[0])
        #headquarters
        if(DF_n['label']=='Headquarters').any() and not isinstance(DF_n.loc[int(np.where(DF_n['label']=='Headquarters')[0])]['data'],list):
            string = DF_n[DF_n['label']=='Headquarters']['data'].values[0]
            str_split = string.split(',')
            #insert list in the dataframe
            DF_n.iat[int(np.where(DF_n['label']=='Headquarters')[0]),DF_n.columns.get_loc('data')] = str_split

        #products
        if(DF_n['label']=='Products').any() and not isinstance(DF_n.loc[int(np.where(DF_n['label']=='Products')[0])]['data'],list):
            string = DF_n[DF_n['label']=='Products']['data'].values[0]
            #split st by a upper case letter
            str_split = re.findall(r'[A-Z][^A-Z]*', string)
            DF_n.iat[int(np.where(DF_n['label']=='Products')[0]),DF_n.columns.get_loc('data')] = str_split

        #division
        if(DF_n['label']=='Divisions').any() and not isinstance(DF_n.loc[int(np.where(DF_n['label']=='Divisions')[0])]['data'],list):
            string = DF_n[DF_n['label']=='Divisions']['data'].values[0]
            #split st by a upper case letter
            str_split = string.split(',')
            DF_n.iat[int(np.where(DF_n['label']=='Divisions')[0]),DF_n.columns.get_loc('data')] = str_split

        #number of employees
        if(DF_n['label']=='Number of employees').any() and not isinstance(DF_n.loc[int(np.where(DF_n['label']=='Number of employees')[0])]['data'],list):
            string = DF_n[DF_n['label']=='Number of employees']['data'].values[0]
            string = string.replace(',','')
            #find all numbers in the string
            numbers = re.findall(r'\d+', string)[0]
            DF_n.iat[int(np.where(DF_n['label']=='Number of employees')[0]),DF_n.columns.get_loc('data')] = numbers

        #networth
        stats = ["Revenue", "Operating income", "Net income", "Total assets", "Total equity"]

        for stat in stats:
            if(DF_n['label']==stat).any() and not isinstance(DF_n.loc[int(np.where(DF_n['label']==stat)[0])]['data'],list):
                string = DF_n[DF_n['label']==stat]['data'].values[0]
                string = string.split(' (')[0]
                DF_n.iat[int(np.where(DF_n['label']==stat)[0]),DF_n.columns.get_loc('data')] = string
                
        #subsidaries
        if(DF_n['label']=='Subsidiaries').any() and not isinstance(DF_n.loc[int(np.where(DF_n['label']=='Subsidiaries')[0])]['data'],list):
            string = DF_n[DF_n['label']=='Subsidiaries']['data'].values[0]
            #split st by a upper case letter
            str_split = string.split(',')
            #insert list in the dataframe
            DF_n.iat[int(np.where(DF_n['label']=='Subsidiaries')[0]),DF_n.columns.get_loc('data')] = str_split

        Dataframes.append(DF_n)
        #Dataframes.append(DF_n)
    except:
        Dataframes.append(pd.DataFrame({'label': ['no info'], 'data': ['missing']}))
    finally:
        #print("The 'try except' is finished")
        next

In [160]:
#json to pandas dataframe
df['Wikipedia Entry'] = Dataframes

In [162]:
#-----Password hidden
f=open("PW.txt", "r") 
userPw=[f.readline()]
f.close()

In [163]:
#------------------------SQL Connection

conn = psycopg2.connect(host="mds-dsi-db.postgres.database.azure.com",
                        port="5432",
                        database="finance_data",
                        user="ds22m017",
                        password=str(userPw[0]),
                        connect_timeout=3)
cur = conn.cursor()

print("con done")

con done


In [164]:
#-----------------------CREATE TABLES


tables = ['wiki_sp_500_companies']
for table in tables:
    cur.execute(f"DROP TABLE IF EXISTS {table} CASCADE;")
    cur.execute(f"CREATE TABLE IF NOT EXISTS {table} (id serial, Symbol VARCHAR(50), Sector VARCHAR(50), info JSONB);")
conn.commit()


print("create done")

create done


In [165]:
for i in range(len(df)):
    sym = df['Symbol'][i]
    sec = df['Sector'][i]
    j_son = df['Wikipedia Entry'][i].set_index('label').transpose().to_json(orient='records').replace('\\u00a0','')[1:-1]
    
    #postgres_insert_query = """ INSERT INTO wiki_sp_500_companies (ID, MODEL, PRICE) VALUES (%s,%s,%s)"""
    cur.execute(f"INSERT INTO wiki_sp_500_companies ( Symbol,Sector, info) VALUES (%s,%s,%s)", [sym,sec, j_son])
conn.commit()

